In [1]:
import random
import itertools
import numpy as np

from collections import Counter

In [2]:
with open('corpora/WarAndPeace.txt', mode='r') as f:
    wap_text = f.read()
    
with open('corpora/AnnaKarenina.txt', mode='r') as f:
    ak_text = f.read()
    
all_text = ' '.join((wap_text, ak_text))

### 1. Реализуйте базовый частотный метод по Шерлоку Холмсу

In [3]:
rus_letters = ''.join([' '] + [chr(i) for i in range(ord('а'), ord('а') + 32)])
rus_letters

' абвгдежзийклмнопрстуфхцчшщъыьэюя'

In [4]:
def clean_text(text):
    text = text.lower().replace('ё', 'е')
    text = ''.join(letter for letter in text if letter in rus_letters)
    text = text.replace('   ', ' ')
    text = text.replace('  ', ' ').replace('  ', ' ')
    return text


def get_frequencies(text):
    return {item[0]:item[1] for item in Counter(text).most_common()}


def simple_encode(text):
    text = clean_text(text)
    letters_map = {letter: new_letter for letter, new_letter
                   in zip(rus_letters, random.sample(rus_letters, len(rus_letters)))}
    return ''.join(letters_map[letter] for letter in text)


def simple_decode(all_text_letters, encoded_text):
    encoded_text_letters = get_frequencies(encoded_text)
    letters_map = {letter: new_letter for letter, new_letter
                  in zip(encoded_text_letters.keys(), all_text_letters.keys())}
    return ''.join(letters_map[letter] for letter in encoded_text)


def accuracy(original_text, decoded_text):
    numerator = 0
    for i in range(len(original_text)):
        if original_text[i] == decoded_text[i]:
            numerator += 1
    denominator = len(original_text) or 1
    return numerator / denominator

In [5]:
# Л. Н. Толстой. Воскресение (отрывок)

test_text = '''В первый раз увидал Нехлюдов Катюшу тогда, когда он на третьем курсе университета, 
готовя свое сочинение о земельной собственности, прожил лето у своих тетушек. 
Обыкновенно он с матерью и сестрой жил летом в материнском большом подмосковном имении. 
Но в этот год сестра его вышла замуж, а мать уехала на воды за границу. 
Нехлюдову же надо было писать сочинение, и он решил прожить лето у тетушек. 
У них в их глуши было тихо, не было развлечений; тетушки же нежно любили своего племянника и наследника, 
и он любил их, любил их старомодность и простоту жизни.'''

test_text = clean_text(test_text)
test_text

'в первый раз увидал нехлюдов катюшу тогда когда он на третьем курсе университета готовя свое сочинение о земельной собственности прожил лето у своих тетушек обыкновенно он с матерью и сестрой жил летом в материнском большом подмосковном имении но в этот год сестра его вышла замуж а мать уехала на воды за границу нехлюдову же надо было писать сочинение и он решил прожить лето у тетушек у них в их глуши было тихо не было развлечений тетушки же нежно любили своего племянника и наследника и он любил их любил их старомодность и простоту жизни'

In [6]:
encoded_test_text = simple_encode(test_text)
encoded_test_text

'бйэътбдуйтсчйяблюсиймъгищюабйпснщ яйнаоюсйпаоюсйамймсйнтънкъжйпятьъйямлбътьлнънсйоанабейьбаъйьашлмъмлъйайчъжъикмауйьафьнбъммаьнлйэтацлийиънайяйьбалгйнъня ъпйафдпмабъммайамйьйжснъткщйлйьъьнтауйцлийиънажйбйжснътлмьпажйфаик ажйэаюжаьпабмажйлжъмллймайбйвнанйоаюйьъьнтсйъоайбд исйчсжяцйсйжснкйяъгсисймсйбаюдйчсйотсмлзяймъгищюабяйцъймсюайфдиайэльснкйьашлмъмлъйлйамйтъ лийэтацлнкйиънайяйнъня ъпйяймлгйбйлгйоия лйфдиайнлгаймъйфдиайтсчбиъшъмлуйнъня плйцъймъцмайищфлилйьбаъоайэиъжеммлпсйлймсьиъюмлпсйлйамйищфлийлгйищфлийлгйьнстажаюмаьнкйлйэтаьнаняйцлчмл'

In [7]:
all_text_letters = get_frequencies(clean_text(all_text))
decoded_test_text = simple_decode(all_text_letters, encoded_test_text)
decoded_test_text

'в зервчш ртй квамтс непсжмов утижьк иогмт уогмт он нт иреиыед укрле кнаверлаиеит гоиовю лвое лоханенае о йедесынош лояливеннолиа зробас сеио к лвоап иеикьеу оячуновенно он л дтиерыж а лелирош бас сеиод в дтиеранлуод яосыьод зомдолуовнод аденаа но в эиои гом лелирт его вчьст йтдкб т дтиы кептст нт вомч йт гртнацк непсжмовк бе нтмо ячсо залтиы лоханенае а он реьас зробаиы сеио к иеикьеу к нап в ап гскьа ячсо иапо не ячсо ртйвсехенаш иеикьуа бе небно сжяаса лвоего зседюннаут а нтлсемнаут а он сжяас ап сжяас ап литродомнолиы а зролиоик байна'

In [8]:
accuracy(test_text, decoded_test_text)

0.49171270718232046

Текст лично для меня нечитаем, хотя доля правильно расшифрованных букв кажется не такой уж и маленькой.

### 2. Реализуйте частотный метод биграмм

In [9]:
def get_bigram_frequencies(text):
    bigrams = [text[i:i + 2] for i in range(0, len(text) - 1, 2)]
    return {item[0]:item[1] for item in Counter(bigrams).most_common()}


def bigram_decode(all_text_bigrams, encoded_text):
    encoded_text_bigrams = get_bigram_frequencies(encoded_test_text)
    bigram_map = {bigram: new_bigram for bigram, new_bigram
                  in itertools.zip_longest(
                      encoded_text_bigrams.keys(), all_text_bigrams.keys(), fillvalue='  '
                  )}
    
    result_text = ''.join([
        bigram_map.get(encoded_text[i:i + 2], encoded_text[i:i + 2])
        for i in range(0, len(encoded_text) - 1, 2)
    ])
    
    if len(encoded_text) % 2 == 1:
        result_text += ' '
    
    return result_text

In [10]:
all_text_bigrams = get_bigram_frequencies(clean_text(all_text))
decoded_test_text = bigram_decode(all_text_bigrams, encoded_test_text)
decoded_test_text

'по ек анкоровориог в пм  бтоилондео  ооти каоти  те и т  иодя атетнавеакдаы сту оспртосеазвиа  зчтов унил  чбыем ра жеь ин гй лоне сал вла овоа раенол итьесни дтитоовелниитдов стасег керь  сльал вла оя пов стобвская ю  лмоя чеавй каойсяно аго келорны осьлимеерь мих лиоримвал с нн экион мивэтвае и рамал и тваямуо  пм  бтоо тре ну н дренеейон мслам пгона к ткорузане салн та и но стсоылвыо гоомпоеносаредиеое нло же на дрекороисхоовшеол итьдиняна пев нсклепеа рача над ч ягоееное болатебеи  к ттаожзаноомскле вена у  сзнтей н ноне сь про ализ '

In [11]:
accuracy(test_text, decoded_test_text)

0.06998158379373849

Текст нечитаем, при этом доля правильно расшифрованных букв сильно упала (примерно в 7 раз). Это вероятно имеет простое объяснение - нам нужен намного более длинный "тестовый" текст, чтобы набрать статистику встречаемости биграмм близкую, к статистике которую мы получаем на большом корпусе текстов ("Война и мир" + "Анна Каренина").

### 3. MCMC-сэмплирование

Можем предположить, что в тексте следующий символ зависит от предыдущего -> можем представить текст как марковскую цепь. На каждой итерации, будем менять символы в нашем словаре случайным образом. Будем рассчитывать правдоподобие для старой версии словаря и для новой. Если для новой версии правдоподобие больше - будем использовать новый словарь. Если нет - воспользуемся новым словарем с вероятностью exp(new_loglikelihood - old_loglikelihood).

In [12]:
class MCMC:
    def __init__(self, train, verbose=True):
        self.compute_train_frequencies(train)
        self.train_letters = [x[0] for x in get_frequencies(train)]
        
        self.verbose = verbose
        
    def compute_train_frequencies(self, train):
        frequencies = get_bigram_frequencies(train)
        sum_all = np.sum(list(frequencies.values()))
        frequencies = {k: (v / (sum_all + 1)) for k, v in frequencies.items()}
        
        self.none_frequency = np.min(list(frequencies.values()))
        self.train_bigram_frequencies = frequencies
        
    def get_map_and_likelyhood(self, train_letters, test_letters, test):
        letters_map = {letter: new_letter for letter, new_letter in zip(test_letters, train_letters)}
        test_decoded = [letters_map[letter] for letter in test]
        
        log_likelyhood = 0
        for i in range(len(test_decoded) - 2 + 1):
            frequency = self.train_bigram_frequencies.get(''.join(test_decoded[i:i + 2]), self.none_frequency)
            log_likelyhood += np.log(frequency)
    
        return letters_map, log_likelyhood
    
    def get_train_letters_transposition(self):
        res = self.train_letters[:]
        idx1, idx2 = np.random.choice(len(res), size=2, replace=False)
        res[idx1], res[idx2] = res[idx2], res[idx1]
        return res

    def fit_predict(self, test, iterations):
        test_letters = [x[0] for x in get_frequencies(test)]
        letters_map, likelyhood = self.get_map_and_likelyhood(self.train_letters, 
                                                                        test_letters,
                                                                        test)
        
        for i in range(iterations):
            temp_train_letters = self.get_train_letters_transposition()
            temp_letters_map, temp_likelyhood = self.get_map_and_likelyhood(temp_train_letters,
                                                                  test_letters,
                                                                  test)
            
            if (temp_likelyhood > likelyhood 
                or np.exp(temp_likelyhood - likelyhood) > random.random()):
                
                self.train_letters = temp_train_letters
                letters_map = temp_letters_map
                likelyhood = temp_likelyhood
                
                if self.verbose:
                    print(f'New best likelyhood: {likelyhood}')
        
        return ''.join(letters_map[letter] for letter in test)

In [13]:
mcmc = MCMC(clean_text(all_text), verbose=False)
test_decoded = mcmc.fit_predict(encoded_test_text, 9000)
test_decoded

'в первый раз увидал нехлюдов катюшу тогда когда он на третьем курсе университета готовя свое сожинение о земельной собственности прочил лето у своих тетушек обыкновенно он с матерью и сестрой чил летом в материнском большом подмосковном имении но в этот год сестра его вышла замуч а мать уехала на воды за границу нехлюдову че надо было писать сожинение и он решил прочить лето у тетушек у них в их глуши было тихо не было развлежений тетушки че нечно любили своего племянника и наследника и он любил их любил их старомодность и простоту чизни'

In [14]:
accuracy(test_text, test_decoded)

0.9797421731123389

Текст стал читаем (практически идеально), доля правильно расшифрованных букв также очень сильно выросла. Кажется, все ок.

### 4. Расшифруйте сообщение

In [15]:
symbols_1 = '←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏'

In [16]:
mcmc = MCMC(clean_text(all_text), verbose=False)
symbols_decoded_1 = mcmc.fit_predict(symbols_1, 90000)
symbols_decoded_1

'если вы вимите норчальный или подти норчальный текст у этого сообщения который легко продитать скорее всего вы все смелали правильно и полудите чаксичальный балл за послемнее детвертое замание курса хотя конедно я нидего не обещаш'

In [17]:
symbols_2 = 'დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ'

In [18]:
mcmc = MCMC(clean_text(all_text), verbose=False)
symbols_decoded_2 = mcmc.fit_predict(symbols_2, 90000)
symbols_decoded_2

'если вы вимите нордальный или почти нордальный текст у этого соожбения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный жалл за послемнее четвертое замание курса хотя конечно я ничего не ожебаф'

Текст в целом читаем. Спасибо за интересное и наглядное упражнение)